In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [13]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [16]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [18]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']), df['has_covid'], test_size=0.2)

In [20]:
x_train

,age,gender,fever,cough,city
76,80,Male,100.0,Mild,Bangalore
24,13,Female,100.0,Strong,Kolkata
89,46,Male,103.0,Strong,Bangalore
77,8,Female,101.0,Mild,Kolkata
49,44,Male,104.0,Mild,Mumbai
...,...,...,...,...,...
10,75,Female,NaN,Mild,Delhi
34,74,Male,102.0,Mild,Mumbai
59,6,Female,104.0,Mild,Kolkata
30,15,Male,101.0,Mild,Delhi


### Without column transformer

In [39]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [38]:
# handling missing values using simple imputer in sklearn.impute for "fever" column
si = SimpleImputer()

x_train_fever = si.fit_transform(x_train[['fever']])
x_test_fever = si.fit_transform(x_test[['fever']])

In [30]:
x_train_fever.shape

(80, 1)

In [40]:
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])

x_train_cough = oe.fit_transform(x_train[['cough']])
x_test_cough = oe.fit_transform(x_test[['cough']])

In [42]:
x_train_cough.shape

(80, 1)

In [43]:
ohe = OneHotEncoder(drop='first', sparse_output=False, dtype=np.int32)

x_train_gender_city = ohe.fit_transform(x_train[['gender', 'city']])
x_test_gender_city = ohe.fit_transform(x_test[['gender', 'city']])

In [45]:
x_train_gender_city.shape

(80, 4)

In [47]:
x_train

,age,gender,fever,cough,city
76,80,Male,100.0,Mild,Bangalore
24,13,Female,100.0,Strong,Kolkata
89,46,Male,103.0,Strong,Bangalore
77,8,Female,101.0,Mild,Kolkata
49,44,Male,104.0,Mild,Mumbai
...,...,...,...,...,...
10,75,Female,NaN,Mild,Delhi
34,74,Male,102.0,Mild,Mumbai
59,6,Female,104.0,Mild,Kolkata
30,15,Male,101.0,Mild,Delhi


In [48]:
x_train_age = x_train.drop(columns=['gender','fever', 'cough', 'city']).values
x_test_age = x_test.drop(columns=['gender','fever', 'cough', 'city']).values


In [49]:
x_train_age.shape

(80, 1)

In [50]:
x_train_transformed = np.concatenate((x_train_age, x_train_fever, x_train_gender_city, x_train_cough), axis=1)
x_test_transformed = np.concatenate((x_test_age, x_test_fever, x_test_gender_city, x_test_cough), axis=1)

In [54]:
x_train_transformed.shape

(80, 7)

### With Column Transformer

In [55]:
from sklearn.compose import ColumnTransformer

In [60]:
trans = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(drop='first', sparse_output=False, dtype=np.int32), ['gender', 'city'])
    ], remainder='passthrough')

In [63]:
trans.fit_transform(x_train).shape

(80, 7)

In [64]:
trans.transform(x_test)

array([[103.,   0.,   0.,   0.,   1.,   0.,  50.],
       [101.,   0.,   0.,   0.,   0.,   0.,  20.],
       [ 98.,   0.,   1.,   0.,   0.,   0.,  73.],
       [101.,   0.,   1.,   1.,   0.,   0.,  42.],
       [102.,   0.,   1.,   0.,   0.,   0.,  64.],
       [104.,   0.,   0.,   0.,   0.,   0.,  18.],
       [100.,   0.,   1.,   0.,   0.,   0.,  10.],
       [100.,   0.,   1.,   1.,   0.,   0.,  27.],
       [ 98.,   1.,   0.,   0.,   0.,   1.,  81.],
       [101.,   1.,   0.,   0.,   1.,   0.,  51.],
       [ 98.,   0.,   1.,   0.,   1.,   0.,  24.],
       [103.,   0.,   0.,   0.,   0.,   0.,  16.],
       [ 98.,   1.,   1.,   0.,   1.,   0.,  34.],
       [100.,   1.,   0.,   0.,   1.,   0.,  11.],
       [104.,   1.,   0.,   0.,   1.,   0.,  54.],
       [102.,   1.,   0.,   0.,   0.,   0.,  24.],
       [ 99.,   0.,   0.,   0.,   0.,   0.,  22.],
       [ 99.,   0.,   1.,   1.,   0.,   0.,  65.],
       [103.,   1.,   1.,   0.,   1.,   0.,  70.],
       [ 99.,   0.,   1.,   0.,